In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

classes = os.listdir('final_dataset/train')
print(classes)

['Adult', 'Airplane', 'Alpaca', 'Bird', 'Bus', 'Car', 'Cat', 'Child', 'Elephant', 'Flower', 'Giraffe', 'Horse', 'Monkey', 'Panda', 'Reptile', 'Vessel']


In [10]:
items = []

for item in classes:
    all_items = os.listdir('final_dataset/train'+'/'+item)
    for itemm in all_items:
        items.append((item, str('final_dataset/train'+'/'+item)+'/'+itemm))


In [11]:
items_df = pd.DataFrame(data = items, columns =['item class','image'])
print(items_df.head())
print(items_df.tail())

  item class                                         image
0      Adult     final_dataset/train/Adult/adults_ (1).jpg
1      Adult    final_dataset/train/Adult/adults_ (10).jpg
2      Adult  final_dataset/train/Adult/adults_ (100).JPEG
3      Adult  final_dataset/train/Adult/adults_ (101).JPEG
4      Adult  final_dataset/train/Adult/adults_ (102).JPEG
     item class                                         image
1833     Vessel  final_dataset/train/Vessel/vessels_ (95).jpg
1834     Vessel  final_dataset/train/Vessel/vessels_ (96).jpg
1835     Vessel  final_dataset/train/Vessel/vessels_ (97).jpg
1836     Vessel  final_dataset/train/Vessel/vessels_ (98).jpg
1837     Vessel  final_dataset/train/Vessel/vessels_ (99).jpg


In [12]:
import cv2
path = 'final_dataset/train'


im_size= 300

images=[]
labels=[]

for i in classes:
    data_path =  path + '/'+str(i)
    filenames=[i for i in os.listdir(data_path)]
    
    for f in filenames:
        img=cv2.imread(data_path+'/'+f)
        img =cv2.resize(img, (im_size,im_size))
        images.append(img)
        labels.append(i)


In [13]:
images = np.array(images)
images.shape

(1838, 300, 300, 3)

In [14]:
images = images.astype('float32')/255.0 #pixel intensity lies between 0 to 255

In [ ]:
images.shape

(1838, 300, 300, 3)

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


y=items_df['item class'].values
print(y[5:])

['Adult' 'Adult' 'Adult' ... 'Vessel' 'Vessel' 'Vessel']


In [16]:
y_labelencoder = LabelEncoder()
y=y_labelencoder.fit_transform(y)
print(y)

[ 0  0  0 ... 15 15 15]


In [17]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, y = shuffle(images,y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.2)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(1470, 300, 300, 3)
(368, 300, 300, 3)
(1470,)
(368,)


In [6]:
import keras
import tensorflow as tf
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(300,300,3)),
    keras.layers.Dense(256, activation=tf.nn.tanh),
    keras.layers.Dense(16, activation=tf.nn.softmax)
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 270000)            0         
                                                                 
 dense (Dense)               (None, 256)               69120256  
                                                                 
 dense_1 (Dense)             (None, 16)                4112      
                                                                 
Total params: 69,124,368
Trainable params: 69,124,368
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, epochs=50)

Epoch 1/50
46/46 [==============================] - 42s 876ms/step - loss: 5.3813 - accuracy: 0.0551
Epoch 2/50
46/46 [==============================] - 40s 863ms/step - loss: 2.8771 - accuracy: 0.0769
Epoch 3/50
46/46 [==============================] - 39s 848ms/step - loss: 2.8071 - accuracy: 0.0646
Epoch 4/50
46/46 [==============================] - 42s 915ms/step - loss: 2.7966 - accuracy: 0.0585
Epoch 5/50
46/46 [==============================] - 42s 904ms/step - loss: 2.8010 - accuracy: 0.0633
Epoch 6/50
46/46 [==============================] - 42s 902ms/step - loss: 2.8024 - accuracy: 0.0476
Epoch 7/50
46/46 [==============================] - 41s 891ms/step - loss: 2.8004 - accuracy: 0.0701
Epoch 8/50
46/46 [==============================] - 41s 886ms/step - loss: 2.7994 - accuracy: 0.0626
Epoch 9/50
46/46 [==============================] - 41s 888ms/step - loss: 2.8031 - accuracy: 0.0585
Epoch 10/50
46/46 [==============================] - 42s 904ms/step - loss: 2.8117 - accura

In [ ]:
y_pred=model.predict(test_x)
y_pred

12/12 [==============================] - 1s 82ms/step


array([[0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ],
       [0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ],
       [0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ],
       ...,
       [0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ],
       [0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ],
       [0.05926634, 0.04852723, 0.09281002, ..., 0.07718588, 0.03549857,
        0.0373262 ]], dtype=float32)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_pred)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [7]:
model2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(300,300,3)),
    keras.layers.Dense(256, activation=tf.nn.tanh),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.softmax)
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 270000)            0         
                                                                 
 dense_2 (Dense)             (None, 256)               69120256  
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
Total params: 69,137,744
Trainable params: 69,137,744
Non-trainable params: 0
_________________________________________________________________


In [18]:
model2.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(train_x, train_y, epochs=50)


y_pred2=model2.predict(test_x)

Epoch 1/50
46/46 [==============================] - 52s 976ms/step - loss: 2.9139 - accuracy: 0.0612
Epoch 2/50
46/46 [==============================] - 42s 923ms/step - loss: 2.8087 - accuracy: 0.0646
Epoch 3/50
46/46 [==============================] - 42s 920ms/step - loss: 2.8013 - accuracy: 0.0735
Epoch 4/50
46/46 [==============================] - 42s 915ms/step - loss: 2.8155 - accuracy: 0.0585
Epoch 5/50
46/46 [==============================] - 42s 911ms/step - loss: 2.7962 - accuracy: 0.0769
Epoch 6/50
46/46 [==============================] - 44s 949ms/step - loss: 2.7913 - accuracy: 0.0714
Epoch 7/50
46/46 [==============================] - 43s 937ms/step - loss: 2.7884 - accuracy: 0.0728
Epoch 8/50
46/46 [==============================] - 41s 881ms/step - loss: 2.7966 - accuracy: 0.0755
Epoch 9/50
46/46 [==============================] - 41s 897ms/step - loss: 2.7857 - accuracy: 0.0864
Epoch 10/50
46/46 [==============================] - 40s 867ms/step - loss: 2.7867 - accura

In [35]:
rand = np.random.RandomState(42)
imgt=cv2.imread('image/aadult.jpg')
imgt =cv2.resize(imgt, (im_size,im_size))
imgt=imgt.reshape((300,300,3))
sample = test_x[1] 
model2.predict(imgt)

ValueError: in user code:

    File "c:\Users\shoba\ML\env\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\shoba\ML\env\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\shoba\ML\env\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\shoba\ML\env\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\shoba\ML\env\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\shoba\ML\env\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "seq